In [7]:
import cv2
import numpy as np
import random

def resize_with_aspect_ratio(image, target_width):
    """Resize the image while keeping aspect ratio based on a target width."""
    h, w = image.shape[:2]
    scale = target_width / w
    target_height = int(h * scale)  # Maintain aspect ratio
    resized_image = cv2.resize(image, (target_width, target_height))
    return resized_image

def remove_background(image):
    # Convert image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define green color range (Adjust these values if needed)
    lower_bound = np.array([25, 40, 40])   # Lower bound for green hues
    upper_bound = np.array([90, 255, 255])  # Upper bound for green hues

    # Create mask for the background (detect non-green areas)
    mask = cv2.inRange(hsv, lower_bound, upper_bound)

    kernel = np.ones((3, 3), np.uint8)
    edges_dilated = cv2.dilate(mask, kernel, iterations=2)

    refined_mask = cv2.bitwise_or(mask, edges_dilated)

    # Extract only the leaf using the mask
    result = cv2.bitwise_and(image, image, mask=refined_mask)

    # Create alpha channel where the leaf is present
    alpha = refined_mask

    # Merge with alpha channel
    b, g, r = cv2.split(result)
    result = cv2.merge((b, g, r, alpha))

    return result


def overlay_image(background, overlay, position, scale=1.0, angle=0):
    """Overlay `overlay` image onto `background` at `position` with given scale and rotation."""
    overlay = remove_background(overlay)

    # Resize the overlay
    h, w = overlay.shape[:2]
    new_w, new_h = int(w * scale), int(h * scale)
    overlay_resized = cv2.resize(overlay, (new_w, new_h))

    # Rotate the overlay
    center = (new_w // 2, new_h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    overlay_resized = cv2.warpAffine(overlay_resized, M, (new_w, new_h))

    # Extract position
    x, y = position
    h_bg, w_bg = background.shape[:2]

    # Ensure overlay fits within background
    if x + new_w > w_bg or y + new_h > h_bg:
        return background  # Skip this overlay if it exceeds bounds

    # Create mask from overlay alpha channel if available
    if overlay_resized.shape[2] == 4:
        alpha = overlay_resized[:, :, 3] / 255.0
        overlay_resized = overlay_resized[:, :, :3]
    else:
        alpha = np.ones((new_h, new_w))  # No transparency

    # Blend images
    for c in range(3):  # RGB channels
        background[y:y+new_h, x:x+new_w, c] = (
            background[y:y+new_h, x:x+new_w, c] * (1 - alpha) +
            overlay_resized[:, :, c] * alpha
        )

    return background

# Load images
vineyard_img = cv2.imread("assets/background_images/IMG_6314.png", cv2.IMREAD_UNCHANGED)  # Load vineyard image
vineyard_img = resize_with_aspect_ratio(vineyard_img, 1200)  # Resize while keeping aspect ratio

grape_leaf_img = cv2.imread("assets/dataset/train/Black Rot/0a06c482-c94a-44d8-a895-be6fe17b8c06___FAM_B.Rot 5019.JPG", cv2.IMREAD_UNCHANGED)  # Load leaf

# Generate random placements
num_leaves = 2  # Change number of leaves as needed
for _ in range(num_leaves):
    x = random.randint(0, vineyard_img.shape[1] - 50)
    y = random.randint(0, vineyard_img.shape[0] - 50)
    scale = random.uniform(1, 2)
    angle = random.randint(0, 360)
    vineyard_img = overlay_image(vineyard_img, grape_leaf_img, (x, y), scale, angle)

# Save or display the result
cv2.imwrite("vineyard_with_leaves.png", vineyard_img)
cv2.imshow("Result", vineyard_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
